In [1]:
# rwr 조화평균 완료

# herb 최종
from tqdm import tqdm_notebook
import numpy as np
import pickle
import pandas as pd
import configparser
import psycopg2
import os
import ast
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
import copy
from collections import defaultdict
from statistics import harmonic_mean

start = time.time()

def calculate_mean(numbers):
    return sum(numbers) / len(numbers)

def calculate_standard_deviation(numbers):
    return statistics.stdev(numbers)

def combine_lists(full_list):
    combined = defaultdict(list)
    
    # 모든 리스트에 대한 처리
    for sublist in full_list:
        for item in sublist:
            entrez_id, rwr_score = item
            combined[entrez_id].append(rwr_score)
    
    # 조화평균 및 결과 리스트 만들기
    result = []
    for entrez_id, scores in combined.items():
        if len(scores) == 1:
            result.append([entrez_id, scores[0]])  # 값이 하나면 그대로 추가
        else:
            # 조화평균 계산
            harmonic_mean_score = harmonic_mean(scores)
            result.append([entrez_id, harmonic_mean_score])
    
    return result

def compound_target(compoundid):

    conn_cur.execute(f"with tmp1 as (select cg.geneid from compound_gene cg where (cg.compid = '{compoundid}' and cg.relationsign in ('activate','inhibit')))" 
                    f"select g.entrezid from gene g,tmp1 where g.geneid=tmp1.geneid;")
    compound_gene1=conn_cur.fetchall()

    compound_gene=set(compound_gene1)
    compound_gene=list(compound_gene)
    
    # Compound-target gene List 생성
    target_gene_list = [] 
    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            target_gene_list.append(interaction_gene)        
        else:
            continue

    # restart node에 적용할 target gene List 생성
    target_node = [] 
    for entrez in target_gene_list:
        node_ = node_index.get(entrez) 
        target_node.append(node_)

    conn_cur.execute(f"with tmp2 as (select cg.geneid from compound_gene cg where cg.compid = '{compoundid}' and relationsign='associate')" 
                    f"select g.entrezid from gene g,tmp2 where g.geneid=tmp2.geneid;")

    compound_gene2=conn_cur.fetchall()

    compound_gene=set(compound_gene2)
    compound_gene=list(compound_gene)

    indirected_target_gene_list = [] 
    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            indirected_target_gene_list.append(interaction_gene)        
        else:
            continue      
    
    indirected_target_gene_list = [gene for gene in indirected_target_gene_list if gene not in target_gene_list]
    
    #indirect nodes 정의
    indirect_nodes= []
    for entrez in indirected_target_gene_list:
        node_ = node_index.get(entrez) 
        indirect_nodes.append(node_)
    
    
    num1=len(target_gene_list) #target gene list 갯수
    directed_target_gene_list.append(num1)
    num2=len(indirected_target_gene_list) #indirected target gene list 갯수 
    indirected_target_gene_list_count.append(num2)

    # PPI network RWR을 위한 initialize
    initialized = RWR_initializing(G, seed_nodes=target_node,indirect_target_nodes=indirect_nodes, is_weighted=False) 

    # RWR 실행
    rwr = RWR(initialized=initialized, prob=0.2, max_iter=100, tol=1.0e-6)
    rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]
    
    var_name1 = f'{compoundid}_num1'
    var_name2 = f'{compoundid}_num2'

    # 전역 변수 설정
    globals()[var_name1] = num1
    globals()[var_name2] = num2
    
    return rwr_mapping_entrez

def rwr_compound_target(count,entrez_list):
    
    num1=directed_target_gene_list[count]
    num2=indirected_target_gene_list_count[count]
    
    random_interaction_gene1 = random.sample(entrez_list, num1)
    remaining_entrez_list = [gene for gene in entrez_list if gene not in random_interaction_gene1]
    random_interaction_gene2 = random.sample(remaining_entrez_list, num2)
    # restart node에 적용할 target gene List 생성
    target_node = [] 
    for entrez in random_interaction_gene1:
        node_ = node_index.get(entrez)
        target_node.append(node_)

    #indirect nodes 정의
    indirect_nodes= []
    for entrez in random_interaction_gene2:
        node_ = node_index.get(entrez)
        indirect_nodes.append(node_)

    # PPI network RWR을 위한 initialize
    initialized = RWR_initializing(G, seed_nodes=target_node,indirect_target_nodes=indirect_nodes, is_weighted=False) 
    
    # RWR 실행
    rwr=RWR(initialized=initialized, prob=0.2, max_iter=100, tol=1.0e-6)
    rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]
    return rwr_mapping_entrez

def create_ppi_network(network_data):
    network_data = network_data.astype({'Entrez Gene Interactor A': str,
                                        'Entrez Gene Interactor B': str})

    # symbol list를 생성하고 unique 값만 가지도록 생성
    symbolA = network_data.loc[:, 'Entrez Gene Interactor A'].to_list()
    symbolB = network_data.loc[:, 'Entrez Gene Interactor B'].to_list()

    symbol_list = symbolA + symbolB 
    symbol_list = set(symbol_list) 
    symbol_list = list(symbol_list)

    # 모든 symbol에 index를 딕셔너리에서 부여
    node_index = {}
    for i in range(len(symbol_list)):
        node_index[symbol_list[i]] = i

    # index로 node_list 생성
    node_list = node_index.values()
    
    # index로 edge_list 생성
    edge_list = network_data[['Entrez Gene Interactor A', 'Entrez Gene Interactor B']].values.tolist()  #
    for i in range(len(edge_list)):
        edge_list[i][0] = node_index.get(edge_list[i][0])
        edge_list[i][1] = node_index.get(edge_list[i][1])
    
    # 무방향 그래프 오브젝트 생성
    G = nx.Graph()

    # 그래프에 노드 추가
    G.add_nodes_from(node_list)

    # 그래프에 엣지 추가
    G.add_edges_from(edge_list)

    print(f'Number of created nodes: {G.number_of_nodes()}')
    print(f'Number of created edges: {G.number_of_edges()}')

    return G, node_index

def RWR_initializing(G, seed_nodes, indirect_target_nodes, is_weighted=False):
    norm_A = np.zeros(shape=(len(G), len(G)))
    if is_weighted:  
        for i, neighbor_dict in G.adjacency():
            for j, v in neighbor_dict.items():
                norm_A[i][j] = v.get("weight", 1 / len(neighbor_dict))
    else:  
        for i, neighbor_dict in G.adjacency():  
            for j, v in neighbor_dict.items():  
                norm_A[i][j] = 1 / len(neighbor_dict)
    
    # seed nodes와 indirect target nodes를 모두 포함한 딕셔너리 생성
    personalization = {node: 1 for node in seed_nodes}
    for node in indirect_target_nodes:
        personalization[node] = 0.3  
        
    r_0 = np.array([personalization.get(n, 0) for n in range(len(G))]) 
    r_c = np.repeat(1 / len(G), len(G)) 
    return {"norm_A": norm_A, "r_0": r_0, "r_c": r_c, "N": len(G)}

def RWR(initialized, prob=0.85, max_iter=100, tol=1.0e-6):
    if initialized is None:
        raise ValueError('initialized information must be required')

    norm_A = initialized['norm_A']  # 인접행렬 A (정규화된)
    r_0 = initialized['r_0']        # 시작 노드
    r_c = initialized['r_c']        # 가중치 행렬
    N = initialized['N']            # 노드의 개수
    for iteration in range(max_iter):
        r_prev = r_c
        r_c = prob * r_c @ norm_A + (1 - prob) * r_0  # RWR algorithm
        err = np.absolute(r_c - r_prev).sum()
        if err < N * tol:
#             print(f'RWR iteration = {iteration + 1}')
#             print('Converged')
            return r_c
#         else:
#             print(f'RWR iteration = {iteration + 1}, Iteration until convergence ...')
#             print(f'{err} -> {N * tol}')
    return "NotConverged"

In [2]:
# postgresql DB 연동
config = configparser.ConfigParser()
config.read('db_config.ini')
conn=psycopg2.connect(host="168.131.30.66", dbname="coconut",user="dbuser",password="jnudl1") #데이터이름변경
conn_cur = conn.cursor()

# BIOGRID-Homo_sapiens Data 기반 PPI Network 구축
network_data = pd.read_csv("/data/home/sss2061/식약처/Data/BIOGRID-ORGANISM-Homo_sapiens.tab3.txt", sep="\t", encoding="cp949")  
G, node_index = create_ppi_network(network_data=network_data)  
entrez_list = list(node_index.keys())
entrez_list.remove('-')

# 네트워크 활용 gene_phenotype 정보 생성
gene_phenotype_file = '/data/home/sss2061/식약처보고서/서울과기대/Data/HPO_gene_phenotype.csv'
if os.path.isfile(gene_phenotype_file): 
    gene_phenotype_df = pd.read_csv(gene_phenotype_file, encoding='UTF-8', converters={'Phenotype': ast.literal_eval}) 
else: 
    gene_phenotype_info = {}
    for g in range(len(entrez_list)):
        conn_cur.execute(f"with tmp1 as (select g.geneid from gene g where g.entrezid = '{entrez_list[g]}'),"
                         f"tmp2 as (select gp.phenid from gene_phenotype gp, tmp1 where gp.geneid = tmp1.geneid)"
                         f"select distinct p.name from phenotype p, tmp2 where p.phenid = tmp2.phenid;")
        # TASS DB 기반 gene-phenotype 정보 추출
        selected_phenotype = conn_cur.fetchall()
        phenotypes = []
        for i in range(len(selected_phenotype)):
             phenotypes.append(selected_phenotype[i][0]) 
        gene_phenotype_info[entrez_list[g]] = phenotypes
        print(f'{g + 1}/{len(entrez_list)}')
    gene_phenotype_mapping = [[entrez_list[i], gene_phenotype_info.get(entrez_list[i])] for i in range(len(entrez_list))]
    gene_phenotype_df = pd.DataFrame(data=gene_phenotype_mapping, columns=['Entrez ID', 'Phenotype'])
    gene_phenotype_df.to_csv(gene_phenotype_file, index=False, encoding='UTF-8')

phenotype_list = []
for phen_list in gene_phenotype_df['Phenotype']:
    for phen in phen_list:
        if phen not in phenotype_list:
            phenotype_list.append(phen)

/tmp/ipykernel_565106/3108670026.py:8: DtypeWarning: Columns (1,2,18) have mixed types. Specify dtype option on import or set low_memory=False.
  network_data = pd.read_csv("/data/home/sss2061/식약처/Data/BIOGRID-ORGANISM-Homo_sapiens.tab3.txt", sep="\t", encoding="cp949")


Number of created nodes: 27421
Number of created edges: 817463


In [3]:
#------------- 1차 추가
number=1
text=["adenine", "canavanin", "malate", "isoleucine", "phenylalanine", "tyrosine", "isocitrate", "glutamine", "tryptophan","L-malate", 
      "theogallin", "trans-aconitate", "cis-aconitate", "threonate", "benzoic acid", "choline", "citric acid", "coumarin", "phloroglucinol", "gallic acid", 
      "citrullin", "pipecolic acid", "vitamin B", "pantothenic acid", "DL-valine", "vanillin", "theophylline", "caffeine", "protocatechuic aldehyde", "adenosine", 
      "EGCG", "gallocatechin", "N-acetylglutamate", "loliolide", "procyanidin B2", "3H-proline", "dihydromyricetin", "beta-alanine betaine", "epicatechin gallate (ECG", "5'-methylthioadenosine", 
      "epiafzelechin", "ferulic acid", "p-coumaric acid", "caffeic acid", "chlorogenic acid", "quercetin", "alpha-isopropylmalate", "neochlorogenic acid", "rutin", "hyperoside", 
      "4-p-coumaroylquinic acid", "astragalin", "quercetin 7-O-glucoside", "p-coumaroylquinic acid", "procyanidin B1"]

Compoundid_list=[]
for compound in text:
    # SQL 쿼리에서 따옴표 이스케이프
#     compound_escaped = compound.replace("'", "''")
    conn_cur.execute("SELECT compid FROM compound WHERE name = %s", (compound,))
    result = conn_cur.fetchone()
    if result:
        Compoundid_list.append(result[0])
    else:
        Compoundid_list.append(None)  # 데이터가 없는 경우 None 추가

if not os.path.exists('result/{}차'.format(number)):
    os.makedirs('result/{}차'.format(number)) 
    
Compound_list=copy.deepcopy(Compoundid_list)
    
#------------- 1차 끝

global directed_target_gene_list
directed_target_gene_list=[]  #사용자가 입력한 compound의 direct target gene 개수가 들어있는 list

global indirected_target_gene_list_count
indirected_target_gene_list_count=[] #사용자가 입력한 compound의 indirect target gene 개수가 들어있는 list

rwr_mapping_entrez_list=[] #사용자가 입력한 compound 별 rwr mapping entrez가 들어있는 list

for c in range(len(Compound_list)):
    
    compoundid1=Compound_list[c]
    
    rwr_mapping_entrez=compound_target(compoundid1)
    rwr_mapping_entrez_list.append(rwr_mapping_entrez)

rwr_mapping_entrez2 = combine_lists(rwr_mapping_entrez_list) #rwr mapping 조화평균 추출

rwr_result = pd.DataFrame(data=rwr_mapping_entrez2, columns=['Entrez ID', 'RWR_score'])
rwr_result['Entrez ID'] = rwr_result['Entrez ID'].astype(int)
rwr_result = pd.merge(rwr_result, gene_phenotype_df, how='outer')
rwr_result = rwr_result.dropna(subset=['RWR_score'])
rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()] = rwr_result['Phenotype'].loc[
    rwr_result['Phenotype'].isnull()].apply(lambda x: [])
rwr_result = rwr_result.sort_values(by='RWR_score', ascending=False)

# phenotype 별 RWR score를 계산하기 위한 딕셔너리 생성
phenotype_rwr_score_dict = {}
for phen in phenotype_list:
    phenotype_rwr_score_dict[phen] = 0    
    
# 각 phenotype 별 RWR score 확인
for i in range(len(rwr_result)):
    rwr_score = rwr_result['RWR_score'].iloc[i]
    if rwr_result['Phenotype'].iloc[i]:
        for phen in rwr_result['Phenotype'].iloc[i]:
            phenotype_rwr_score_dict[phen] += rwr_score

temp=list(phenotype_rwr_score_dict.keys())
for i in temp:
    if np.isnan(phenotype_rwr_score_dict[i])==True:
        phenotype_rwr_score_dict.pop(i)
        
Phenotype_score_Phenotype=list(phenotype_rwr_score_dict.keys())
Phenotype_score_Score=list(phenotype_rwr_score_dict.values())

#network score. csv 파일로 저장
inferred_phenotype2 = pd.DataFrame(data=[pair for pair in zip(list(phenotype_rwr_score_dict.keys()),
                                                             list(phenotype_rwr_score_dict.values()))],
                                  columns=['Phenotype', 'Network_score'])
inferred_phenotype2 = inferred_phenotype2.sort_values(by='Network_score', ascending=False)
inferred_phenotype2.to_csv('result/{}차/{}_network_score.csv'.format(number,number),index=False, encoding='UTF-8')

In [4]:
output_dict={}

In [5]:
# import pickle
# with open("result/{}차/output_dict.pickle".format(number),"rb") as fi:
#     output_dict = pickle.load(fi)

In [6]:
#----------------------------------------------------------------1,000
for h in tqdm_notebook(range(1000)):
    p_rwr_mapping_entrez_list=[] #1번당 compound별 random target gene rwr mapping list
    
    print(h,"번째 상황") # 제거
    for k in tqdm_notebook(range(len(Compound_list))):
        rwr_mapping_entrez=rwr_compound_target(k,entrez_list)
        p_rwr_mapping_entrez_list.append(rwr_mapping_entrez)
    
    rwr_mapping_entrez3 = combine_lists(p_rwr_mapping_entrez_list) #rwr mapping 조화평균 추출
    
    rwr_result = pd.DataFrame(data=rwr_mapping_entrez3, columns=['Entrez ID', 'RWR_score'])
    rwr_result['Entrez ID'] = rwr_result['Entrez ID'].astype(int)
    rwr_result = pd.merge(rwr_result, gene_phenotype_df, how='outer')
    rwr_result = rwr_result.dropna(subset=['RWR_score'])
    rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()] = rwr_result['Phenotype'].loc[
        rwr_result['Phenotype'].isnull()].apply(lambda x: [])
    rwr_result = rwr_result.sort_values(by='RWR_score', ascending=False)
    
# phenotype 별 RWR score를 계산하기 위한 딕셔너리 생성
    phenotype_rwr_score_dict = {}
    for phen in phenotype_list:
        phenotype_rwr_score_dict[phen] = 0

# 각 phenotype 별 RWR score 확인
    for i in range(len(rwr_result)):
        rwr_score = rwr_result['RWR_score'].iloc[i]
        if rwr_result['Phenotype'].iloc[i]:
            for phen in rwr_result['Phenotype'].iloc[i]:
                phenotype_rwr_score_dict[phen] += rwr_score

    key_list=list(phenotype_rwr_score_dict.keys())
    for i in key_list:
        if i not in output_dict:
            output_dict[i]=list([phenotype_rwr_score_dict[i]])
        else:
            output_dict[i].append(phenotype_rwr_score_dict[i])
    
    with open("result/{}차/output_dict.pickle".format(number),"wb") as f:
        pickle.dump(output_dict,f)
    
print("완료1")

temp2=list(output_dict.keys())
for i in temp2:
    if np.isnan(output_dict[i][0])==True:
        output_dict.pop(i)
        
#rwr. score csv 파일로 저장
inferred_phenotype3 = pd.DataFrame(data=[pair for pair in zip(list(output_dict.keys()),
                                                             list(output_dict.values()))],
                                  columns=['Phenotype', 'Network_score'])
inferred_phenotype3 = inferred_phenotype3.sort_values(by='Network_score', ascending=False)

inferred_phenotype3.to_csv('result/{}차/{}_P_value.csv'.format(number,number),index=False, encoding='UTF-8')

P_value_Phenotype=list(output_dict.keys())
P_value_Score=list(output_dict.values())

/tmp/ipykernel_565106/3176965269.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for h in tqdm_notebook(range(1000)):


  0%|          | 0/1000 [00:00<?, ?it/s]

0 번째 상황


/tmp/ipykernel_565106/3176965269.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for k in tqdm_notebook(range(len(Compound_list))):


  0%|          | 0/55 [00:00<?, ?it/s]

...

  0%|          | 0/55 [00:00<?, ?it/s]

999 번째 상황


/tmp/ipykernel_565106/3176965269.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for k in tqdm_notebook(range(len(Compound_list))):


  0%|          | 0/55 [00:00<?, ?it/s]

완료1


In [7]:
print(len(output_dict['Autosomal recessive inheritance']))

1000


In [8]:
#----------------------------------------------------------------z-score  
count=0
z_score_list=[]
outlier_list=[] #제거가능
P_value_list=[]
rank_number=0
result_phenotype=[]
result_phenotype_score=[]

for i in range(len(P_value_Score)):
    Numberlist=P_value_Score[i].copy()
    tmp1=P_value_Phenotype[i]
    tmp2=Phenotype_score_Phenotype.index(tmp1)
    tmp3=Phenotype_score_Score[tmp2]
    Numberlist.append(tmp3)
    result_phenotype.append(tmp1)
    result_phenotype_score.append(tmp3)
    
    # 평균 계산
    mean = calculate_mean(Numberlist)
    # 표준편차 계산
    std_deviation = calculate_standard_deviation(Numberlist)
    if std_deviation==0:    #제거
        z_score=0
    else:
        z_score=(tmp3-mean)/std_deviation

    z_score_list.append(z_score)
    if z_score >=-2.58 and z_score<=2.58: #제거가능
        outlier=1
        count=count+1
    else:
        outlier=0 
    outlier_list.append(outlier) #제거가능

    Numberlist.sort(reverse=True)

    rank=Numberlist.index(tmp3) #순위찾기
    if rank<5:
        rank_number=rank_number+1
    P_value_list.append(rank)

#------------------------

Phenotype_score = pd.DataFrame({
    'Phenotype': result_phenotype,
    'Network_score': result_phenotype_score,
    'z-score': z_score_list,
    'P-value': P_value_list,
})

#     # Compound ID에 해당하는 inferred phenotype.csv 생성
# if not os.path.exists('compound 결과'):
#     os.makedirs('compound 결과')
Phenotype_score.to_csv('result/{}차/{}_검증.csv'.format(number,number),index=False, encoding='UTF-8')
    
end = time.time()
sec = (end - start)
result = datetime.timedelta(seconds=sec)
print("걸린 시간 : ",result)
print(f"걸린 시간 : {end - start:.5f} sec")

걸린 시간 :  3 days, 16:47:45.826003
걸린 시간 : 319665.82600 sec


In [9]:
# 서울과기대 요청 추가 
# 데이터 후처리
# 기존 111번까지는 phenotype / score / z_score / p_value 파일 존재
# 이후 score scaling
# 이후 p_value 0.001 ~ 1
# 이후 "수정_id2" 파일을 통해 phenotype 컬럼을 기준으로 일치하면 umlsid, hpoid 컬럼 추가
# 이후 "phenotype mappingv4" 파일을 통해 umlis 컬럼을 기준으로 일치하면 group, groupid 추가
#------------------------------------
# 'Network_score' 컬럼의 데이터를 추출합니다.
network_scores = Phenotype_score['Network_score']

# Min-Max Scaling을 0부터 1로 설정하여 적용합니다.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_scores = scaler.fit_transform(network_scores.values.reshape(-1, 1))  # 2D 배열로 변환 후 스케일링

# 스케일링된 값을 'scaling_score' 컬럼에 추가합니다.
Phenotype_score['scaling_score'] = scaled_scores

# 'P-value'의 값을 조정합니다.
def adjust_pvalue(value):
    if value == 999 or value == 1000:
        return 1
    else:
        return (value+1) / 1000

Phenotype_score['P-value'] = Phenotype_score['P-value'].apply(adjust_pvalue)


#------------------------------------
#umls db에서 umls id와 phenotype score 파일을 합치는 코드

file_2= pd.read_csv("Data/수정_Id2.csv")

# file과 file_2를 'Phenotype'과 'phenotype' 컬럼을 기준으로 결합합니다.
# 이때 file_2의 'umlsid' 컬럼만 선택하여 추가합니다.
file_3 = pd.merge(Phenotype_score, file_2[['umlsid', 'phenotype','hpoid']], left_on='Phenotype', right_on='phenotype', how='left')

# 중복된 'phenotype' 컬럼을 제거합니다.
file_3.drop(columns=['phenotype'], inplace=True)
Phenotype_score=file_3
# print(Phenotype_score)
#------------------------------------
# 'phenotype mappingv3.xlsx' 파일을 읽어옵니다.
mapping_df = pd.read_excel('Data/phenotype mappingv4.xlsx', engine='openpyxl')

# '1_검증.csv' 파일 내의 UMLSID와 'phenotype mappingv3.xlsx' 파일 내의 UMLSID를 기반으로 병합합니다.
grouped_mapping_df = mapping_df.groupby('UMLSID')['Group'].agg(lambda x: ','.join(x)).reset_index()

# Phenotype_score와 grouped_mapping_df를 병합합니다.
final_df = pd.merge(Phenotype_score, grouped_mapping_df, left_on='umlsid', right_on='UMLSID', how='left')

# 필요없는 UMLSID 컬럼을 제거합니다.
final_df.drop(columns=['UMLSID'], inplace=True)

cols_to_drop = [col for col in final_df.columns if col.startswith("Unnamed")]
final_df = final_df.drop(columns=cols_to_drop)

final_df.to_csv('result/{}차/{}_검증_with_scaling.csv'.format(number,number), index=False) 

In [ ]:
# number=1
# text=["adenine", "canavanin", "malate", "isoleucine", "phenylalanine", "tyrosine", "isocitrate", "glutamine", "tryptophan","L-malate", 
#       "theogallin", "trans-aconitate", "cis-aconitate", "threonate", "benzoic acid", "choline", "citric acid", "coumarin", "phloroglucinol", "gallic acid", 
#       "citrullin", "pipecolic acid", "vitamin B", "pantothenic acid", "DL-valine", "vanillin", "theophylline", "caffeine", "protocatechuic aldehyde", "adenosine", 
#       "EGCG", "gallocatechin", "N-acetylglutamate", "loliolide", "procyanidin B2", "3H-proline", "dihydromyricetin", "beta-alanine betaine", "epicatechin gallate (ECG", "5'-methylthioadenosine", 
#       "epiafzelechin", "ferulic acid", "p-coumaric acid", "caffeic acid", "chlorogenic acid", "quercetin", "alpha-isopropylmalate", "neochlorogenic acid", "rutin", "hyperoside", 
#       "4-p-coumaroylquinic acid", "astragalin", "quercetin 7-O-glucoside", "p-coumaroylquinic acid", "procyanidin B1"]

# number=2
# text=["gamma-aminobutyric acid (GABA", "canavanin", "sucrose", "folate", "guanosine", "vanillic acid", "ononin", 
#       "formononetin", "calycosin", "sissotrin", "afrormosin", "pratensein"]

# number=3
# text=["protocatechuic acid", "gamma-aminobutyric acid (GABA", "choline", "citric acid", "malate", "asparagine", 
#       "quinic acid", "maltol", "nodakenetin", "nodakenin", "columbianetin", "decursin", "ferulic acid", "caffeic acid",
#       "aegelinol", "cis-chlorogenic acid", "chlorogenic acid", "esculetin"]

# number=4
# text=["piperine", "piperanine", "piperlonguminine", "retrofractamide B", "dehydropipernonaline", "Methyl piperate", "pipernonaline", "Retrofractamide A", "retrofractamide C"]

# number=5
# text=["apocynin", "resacetophenone", "4-hydroxyacetophenone", "2,5-dihydroxyacetophenone", "physcion"]

# number=6
# text=["protocatechuic acid", "p-hydroxybenzaldehyde", "choline", "gluconate", "myo-inositol", "thymine", "uracil", "vanillin", "vanillic acid", "protocatechuic aldehyde", "trans-aconitate", "ferulic acid", "caffeic acid", "chlorogenic acid", "linoleic acid", "neochlorogenic acid", "xanthatin", "cynarin", "cynarine"]

# number=7
# text=["atractylenolide III", "vanillic acid", "esculetin", "elemicin", "adenosine"]


In [ ]:
# gene의 연관성을 homo sapiens dataset에서 찾으시는거라면 큰 문제가 없을 것 같지만, compound가 homo sapiens에게 영향을 주는 gene만을 찾으시려면 ncbitaxid가 9606인 gene만 입력되도록 코드를 추가하시는것도 좋을 것 같습니다.